In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import tree

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [63]:
# Binary Classification - text description (magnitudes & scores)
# Created two different models based on text description magnitudes and scores - evaluate emotion level of text

train = pd.read_csv('train.csv') 
train_adoption_speed_old = train['AdoptionSpeed']
train_type = train['Type']

train_adoption_speed = []
for i in train_adoption_speed_old:
    if i <= 3:
        train_adoption_speed.append(0)
    else:
        train_adoption_speed.append(1)

data = {}
train_magnitudes = [];
train_scores = [];
test_magnitudes = [];
test_scores = [];

for root, dirs, files in os.walk("train_sentiment"):
    for filename in files:
        if filename.endswith(".json"):
            data_file_name = 'train_sentiment/' + filename;
            data = json.load(open(data_file_name, 'r'))
            train_magnitudes.append(data["documentSentiment"]["magnitude"] * 1000)
            train_scores.append(data["documentSentiment"]["score"] * 1000)
            
df_train_magnitudes = pd.DataFrame(train_magnitudes)
df_train_scores = pd.DataFrame(train_scores)
df_train_adoption_speed = pd.DataFrame(train_adoption_speed)
df_train_type = pd.DataFrame(train_type)

horizontal_train_df = pd.concat([df_train_magnitudes, df_train_scores, df_train_type], axis=1)
horizontal_train_df.columns = ['train_magnitudes', 'train_scores', 'train_adoption_speed', 'train_type']
horizontal_train_df.dropna(inplace = True)

features_train = ['train_magnitudes', 'train_scores']

X = horizontal_train_df.loc[:, features_train]
y = horizontal_train_df['train_adoption_speed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

clf = LogisticRegression(random_state=0, solver='newton-cg', multi_class='multinomial').fit(X_train, y_train)
print("The score of the model is (TRAIN): %f" % clf.score(X_train, y_train))

y_pred = clf.predict(X_test)
d = {'predictions':y_pred,'true':y_test}
df = pd.DataFrame(d)
print("The score of the model is (TEST): %f" % clf.score(X_test, y_test))

# Decision trees, SVM, Linear SVM, Nearest Neighbors, Logistic Regression

The score of the model is (TRAIN): 0.721523
The score of the model is (TEST): 0.720270


In [64]:
# Decision trees
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
print("The score of the model is (TRAIN): %f" % clf.score(X_train, y_train))

y_pred = clf.predict(X_test)
d = {'predictions':y_pred,'true':y_test}
df = pd.DataFrame(d)
print("The score of the model is (TEST): %f" % clf.score(X_test, y_test))

The score of the model is (TRAIN): 0.737796
The score of the model is (TEST): 0.703999


In [67]:
# Binary Classification - text description (magnitudes & scores) to predict dog vs. cat

train = pd.read_csv('train.csv') 
train_type = train['Type']

data = {}
train_magnitudes = [];
train_scores = [];
test_magnitudes = [];
test_scores = [];

for root, dirs, files in os.walk("train_sentiment"):
    for filename in files:
        if filename.endswith(".json"):
            data_file_name = 'train_sentiment/' + filename;
            data = json.load(open(data_file_name, 'r'))
            train_magnitudes.append(data["documentSentiment"]["magnitude"] * 1000)
            train_scores.append(data["documentSentiment"]["score"] * 1000)
            
df_train_magnitudes = pd.DataFrame(train_magnitudes)
df_train_scores = pd.DataFrame(train_scores)
df_train_type = pd.DataFrame(train_type)

horizontal_train_df = pd.concat([df_train_magnitudes, df_train_scores, df_train_type], axis=1)
horizontal_train_df.columns = ['train_magnitudes', 'train_scores', 'train_type']
horizontal_train_df.dropna(inplace = True)

features_train = ['train_magnitudes','train_scores']

X = horizontal_train_df.loc[:, features_train]
y = horizontal_train_df['train_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

clf = LogisticRegression(random_state=0, solver='newton-cg', multi_class='ovr').fit(X_train, y_train)
print("The score of the model is (TRAIN): %f" % clf.score(X_train, y_train))

y_pred = clf.predict(X_test)
d = {'predictions':y_pred,'true':y_test}
df = pd.DataFrame(d)
print("The score of the model is (TEST): %f" % clf.score(X_test, y_test))

# Decision trees, SVM, Linear SVM, Nearest Neighbors, Logistic Regression

The score of the model is (TRAIN): 0.541373
The score of the model is (TEST): 0.543881


In [ ]:
# Binary Classification - dog vs. cat to predict adoption speed

train = pd.read_csv('train.csv') 
train.dropna(inplace = True)
train_type = train['Type']
train_adoption_speed = train['AdoptionSpeed']

df_train_type = pd.DataFrame(train_type)
df_train_adoption_speed = pd.DataFrame(train_adoption_speed)


horizontal_train_df = pd.concat([df_train_type, df_train_adoption_speed], axis=1)
horizontal_train_df.columns = ['df_train_type', 'df_train_adoption_speed']
horizontal_train_df.dropna(inplace = True)

features_train = ['df_train_type']

X = horizontal_train_df.loc[:, features_train]
y = horizontal_train_df['df_train_adoption_speed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

clf = LogisticRegression(random_state=0, solver='newton-cg', multi_class='ovr').fit(X_train, y_train)
print("The score of the model is (TRAIN): %f" % clf.score(X_train, y_train))

y_pred = clf.predict(X_test)
d = {'predictions':y_pred,'true':y_test}
df = pd.DataFrame(d)
print("The score of the model is (TEST): %f" % clf.score(X_test, y_test))

# Decision trees, SVM, Linear SVM, Nearest Neighbors, Logistic Regression